In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import random 
 
xmlfilepath=r'/content/drive/My Drive/yolov4-pytorch-master/VOCdevkit/VOC2007/Annotations'
saveBasePath=r"/content/drive/My Drive/yolov4-pytorch-master/VOCdevkit/VOC2007/ImageSets/Main"
 
trainval_percent=1
train_percent=1

temp_xml = os.listdir(xmlfilepath)
total_xml = []
for xml in temp_xml:
    if xml.endswith(".xml"):
        total_xml.append(xml)

num=len(total_xml)  
list=range(num)  
tv=int(num*trainval_percent)  
tr=int(tv*train_percent)  
trainval= random.sample(list,tv)  
train=random.sample(trainval,tr)  
 
print("train and val size",tv)
print("traub suze",tr)
ftrainval = open(os.path.join(saveBasePath,'trainval.txt'), 'w')  
ftest = open(os.path.join(saveBasePath,'test.txt'), 'w')  
ftrain = open(os.path.join(saveBasePath,'train.txt'), 'w')  
fval = open(os.path.join(saveBasePath,'val.txt'), 'w')  
 
for i  in list:  
    name=total_xml[i][:-4]+'\n'  
    if i in trainval:  
        ftrainval.write(name)  
        if i in train:  
            ftrain.write(name)  
        else:  
            fval.write(name)  
    else:  
        ftest.write(name)  
  
ftrainval.close()  
ftrain.close()  
fval.close()  
ftest .close()

train and val size 1547
traub suze 1547


In [ ]:
import xml.etree.ElementTree as ET
from os import getcwd

sets=[('2007', 'train'), ('2007', 'val'), ('2007', 'test')]

wd = getcwd()
classes = ['unwear',
'wear',
'uncorrect',
'no_wear']

def convert_annotation(year, image_id, list_file):
    in_file = open('/content/drive/My Drive/yolov4-pytorch-master/VOCdevkit/VOC%s/Annotations/%s.xml'%(year, image_id))
    tree=ET.parse(in_file)
    root = tree.getroot()
    filename = root.find('filename').text
    list_file.write('/content/drive/My Drive/yolov4-pytorch-master/VOCdevkit/VOC%s/JPEGImages/'%(year) + filename)
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if int(difficult)==1:
            continue
        # cls_id = classes.index(cls)
        cls_id = int(cls)
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

    list_file.write('\n')

for year, image_set in sets:
    image_ids = open('/content/drive/My Drive/yolov4-pytorch-master/VOCdevkit/VOC%s/ImageSets/Main/%s.txt'%(year, image_set)).read().strip().split()
    list_file = open('/content/drive/My Drive/yolov4-pytorch-master/%s_%s.txt'%(year, image_set), 'w')
    for image_id in image_ids:
        convert_annotation(year, image_id, list_file)
    list_file.close()


In [ ]:
!git clone https://github.com/ultralytics/yolov3 && cd yolov3

Cloning into 'yolov3'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 9345 (delta 5), reused 12 (delta 4), pack-reused 9327
Receiving objects: 100% (9345/9345), 7.18 MiB | 8.26 MiB/s, done.
Resolving deltas: 100% (6419/6419), done.


In [ ]:
import sys
FOLDER_PATH = '/content/yolov3'
sys.path.append(FOLDER_PATH)

In [ ]:
import sys
FOLDER_PATH = '/content/yolov3'
sys.path.append(FOLDER_PATH)
!python3  -c "import sys; FOLDER_PATH = '/content/yolov3'; sys.path.append(FOLDER_PATH); from models import *; convert('/content/yolov3/cfg/yolov4.cfg', '/content/drive/My Drive/yolov4-pytorch-master/model_data/yolov4.weights')"

Model Summary: 327 layers, 6.43631e+07 parameters, 6.43631e+07 gradients
Success: converted '/content/drive/My Drive/yolov4-pytorch-master/model_data/yolov4.weights' to '/content/drive/My Drive/yolov4-pytorch-master/model_data/yolov4.pt'


In [1]:
import sys
FOLDER_PATH = '/content/drive/My Drive/yolov4-pytorch-master'
sys.path.append(FOLDER_PATH)

#-------------------------------------#
#       对数据集进行训练
#-------------------------------------#
import os
import numpy as np
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from utils.dataloader import yolo_dataset_collate, YoloDataset
from nets.yolo_training import YOLOLoss,Generator
from nets.yolo4 import YoloBody


#---------------------------------------------------#
#   获得类和先验框
#---------------------------------------------------#
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape([-1,3,2])[::-1,:,:]

def fit_ont_epoch(net,yolo_losses,epoch,epoch_size,epoch_size_val,gen,genval,Epoch,cuda):
    total_loss = 0
    val_loss = 0
    start_time = time.time()
    for iteration, batch in enumerate(gen):
        if iteration >= epoch_size:
            break
        images, targets = batch[0], batch[1]
        with torch.no_grad():
            if cuda:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor)).cuda()
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
            else:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor))
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
        optimizer.zero_grad()
        outputs = net(images)
        losses = []
        for i in range(3):
            loss_item = yolo_losses[i](outputs[i], targets)
            losses.append(loss_item[0])
        loss = sum(losses)
        loss.backward()
        optimizer.step()

        total_loss += loss
        waste_time = time.time() - start_time
        if iteration % 100 == 0:
          print('Epoch:'+ str(epoch+1) + '/' + str(Epoch))
          print('iter:' + str(iteration) + '/' + str(epoch_size) + ' || Total Loss: %.4f || %.4fs/step' % (total_loss/(iteration+1),waste_time))
        start_time = time.time()

    print('Start Validation')
    for iteration, batch in enumerate(genval):
        if iteration >= epoch_size_val:
            break
        images_val, targets_val = batch[0], batch[1]

        with torch.no_grad():
            if cuda:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor)).cuda()
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            else:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor))
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            optimizer.zero_grad()
            outputs = net(images_val)
            losses = []
            for i in range(3):
                loss_item = yolo_losses[i](outputs[i], targets_val)
                losses.append(loss_item[0])
            loss = sum(losses)
            val_loss += loss
    print('Finish Validation')
    print('\nEpoch:'+ str(epoch+1) + '/' + str(Epoch))
    print('Total Loss: %.4f || Val Loss: %.4f ' % (total_loss/(epoch_size+1),val_loss/(epoch_size_val+1)))

    return val_loss/(epoch_size_val+1)


if __name__ == "__main__":
    #-------------------------------#
    #   输入的shape大小
    #   显存比较小可以使用416x416
    #   显存比较大可以使用608x608
    #-------------------------------#
    input_shape = (608,608)
    #-------------------------------#
    #   tricks的使用设置
    #-------------------------------#
    Cosine_lr = True
    mosaic = True
    # 用于设定是否使用cuda
    Cuda = True
    smoooth_label = 0.2
    #-------------------------------#
    #   Dataloder的使用
    #-------------------------------#
    Use_Data_Loader = True

    annotation_path = '/content/drive/My Drive/yolov4-pytorch-master/2007_train.txt'
    #-------------------------------#
    #   获得先验框和类
    #-------------------------------#
    anchors_path = '/content/drive/My Drive/yolov4-pytorch-master/model_data/yolo_anchors.txt'
    classes_path = '/content/drive/My Drive/yolov4-pytorch-master/model_data/voc_classes.txt'   
    class_names = get_classes(classes_path)
    anchors = get_anchors(anchors_path)
    num_classes = len(class_names)
    
    # 创建模型
    model = YoloBody(len(anchors[0]),num_classes)
    #-------------------------------------------#
    #   权值文件的下载请看README
    #-------------------------------------------#
    model_path = "/content/drive/My Drive/yolov4-pytorch-master/model_data/yolo4_voc_weights.pth"
    # 加快模型训练的效率
    print('Loading weights into state dict...')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_dict = model.state_dict()
    print(model_dict.keys())
    pretrained_dict = torch.load(model_path, map_location=device)
    print(pretrained_dict.keys())
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if np.shape(model_dict[k]) ==  np.shape(v)}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    print('Finished!')

    net = model.train()

    if Cuda:
        net = torch.nn.DataParallel(model)
        cudnn.benchmark = True
        net = net.cuda()

    # 建立loss函数
    yolo_losses = []
    for i in range(3):
        yolo_losses.append(YOLOLoss(np.reshape(anchors,[-1,2]),num_classes, \
                                (input_shape[1], input_shape[0]), smoooth_label, Cuda))

    # 0.1用于验证，0.9用于训练
    val_split = 0.1
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val
    
    if True:
        lr = 1e-3
        Batch_size = 4
        Init_Epoch = 0
        Freeze_Epoch = 25
        
        optimizer = optim.Adam(net.parameters(),lr,weight_decay=5e-4)
        if Cosine_lr:
            lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-5)
        else:
            lr_scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1,gamma=0.9)
        print('create dataset...')
        if Use_Data_Loader:
            train_dataset = YoloDataset(lines[:num_train], (input_shape[0], input_shape[1]), mosaic=mosaic)
            val_dataset = YoloDataset(lines[num_train:], (input_shape[0], input_shape[1]), mosaic=False)
            gen = DataLoader(train_dataset, batch_size=Batch_size, num_workers=8, pin_memory=True,
                                    drop_last=True, collate_fn=yolo_dataset_collate)
            gen_val = DataLoader(val_dataset, batch_size=Batch_size, num_workers=8,pin_memory=True, 
                                    drop_last=True, collate_fn=yolo_dataset_collate)
        else:
            gen = Generator(Batch_size, lines[:num_train],
                            (input_shape[0], input_shape[1])).generate(mosaic = mosaic)
            gen_val = Generator(Batch_size, lines[num_train:],
                            (input_shape[0], input_shape[1])).generate(mosaic = False)

        epoch_size = max(1, num_train//Batch_size)
        epoch_size_val = num_val//Batch_size
        #------------------------------------#
        #   冻结一定部分训练
        #------------------------------------#
        for param in model.backbone.parameters():
            param.requires_grad = False
        best_loss = 1000
        for epoch in range(Init_Epoch,Freeze_Epoch):
            val_loss = fit_ont_epoch(net,yolo_losses,epoch,epoch_size,epoch_size_val,gen,gen_val,Freeze_Epoch,Cuda)
            lr_scheduler.step()
            if val_loss < best_loss:
              print('Saving state, iter:', str(epoch+1))
              torch.save(model.state_dict(), '/content/drive/My Drive/yolov4-pytorch-master/logs/Epoch%d-Val_Loss%.4f.pth'%((epoch+1),val_loss))
              best_loss = val_loss


    if True:
        lr = 1e-4
        Batch_size = 2
        Freeze_Epoch = 25
        Unfreeze_Epoch = 50

        optimizer = optim.Adam(net.parameters(),lr,weight_decay=5e-4)
        if Cosine_lr:
            lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-5)
        else:
            lr_scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1,gamma=0.9)

        if Use_Data_Loader:
            train_dataset = YoloDataset(lines[:num_train], (input_shape[0], input_shape[1]), mosaic=mosaic)
            val_dataset = YoloDataset(lines[num_train:], (input_shape[0], input_shape[1]), mosaic=False)
            gen = DataLoader(train_dataset, batch_size=Batch_size, num_workers=8, pin_memory=True,
                                    drop_last=True, collate_fn=yolo_dataset_collate)
            gen_val = DataLoader(val_dataset, batch_size=Batch_size, num_workers=8,pin_memory=True, 
                                    drop_last=True, collate_fn=yolo_dataset_collate)
        else:
            gen = Generator(Batch_size, lines[:num_train],
                            (input_shape[0], input_shape[1])).generate(mosaic = mosaic)
            gen_val = Generator(Batch_size, lines[num_train:],
                            (input_shape[0], input_shape[1])).generate(mosaic = False)

        epoch_size = max(1, num_train//Batch_size)
        epoch_size_val = num_val//Batch_size
        #------------------------------------#
        #   解冻后训练
        #------------------------------------#
        for param in model.backbone.parameters():
            param.requires_grad = True

        for epoch in range(Freeze_Epoch,Unfreeze_Epoch):
            fit_ont_epoch(net,yolo_losses,epoch,epoch_size,epoch_size_val,gen,gen_val,Unfreeze_Epoch,Cuda)
            lr_scheduler.step()
            if val_loss < best_loss:
              print('Saving state, iter:', str(epoch+1))
              torch.save(model.state_dict(), '/content/drive/My Drive/yolov4-pytorch-master/logs/Epoch%d-Val_Loss%.4f.pth'%((epoch+1),val_loss))
              best_loss = val_loss

Loading weights into state dict...
odict_keys(['backbone.conv1.conv.weight', 'backbone.conv1.bn.weight', 'backbone.conv1.bn.bias', 'backbone.conv1.bn.running_mean', 'backbone.conv1.bn.running_var', 'backbone.conv1.bn.num_batches_tracked', 'backbone.stages.0.downsample_conv.conv.weight', 'backbone.stages.0.downsample_conv.bn.weight', 'backbone.stages.0.downsample_conv.bn.bias', 'backbone.stages.0.downsample_conv.bn.running_mean', 'backbone.stages.0.downsample_conv.bn.running_var', 'backbone.stages.0.downsample_conv.bn.num_batches_tracked', 'backbone.stages.0.split_conv0.conv.weight', 'backbone.stages.0.split_conv0.bn.weight', 'backbone.stages.0.split_conv0.bn.bias', 'backbone.stages.0.split_conv0.bn.running_mean', 'backbone.stages.0.split_conv0.bn.running_var', 'backbone.stages.0.split_conv0.bn.num_batches_tracked', 'backbone.stages.0.split_conv1.conv.weight', 'backbone.stages.0.split_conv1.bn.weight', 'backbone.stages.0.split_conv1.bn.bias', 'backbone.stages.0.split_conv1.bn.running_me